In [3]:
import importlib
from typing import Sequence
from datasets.dataset_dict import DatasetDict
import fire
import datasets
import sys
from collections import defaultdict

sys.path.append("/home/meissner/shortcut-pruning")
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from tqdm import tqdm
import torch 
from matplotlib import pyplot as plt
from src.models.hf_model import SequenceClassificationTransformer
from src.models.hf_model_pruned import PruningTransformer

import torch
from transformers import AutoModelForSequenceClassification
from nn_pruning.patch_coordinator import ModelPatchingCoordinator
from nn_pruning.modules.masked_nn import MaskedLinear

In [4]:
old = "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/baselines/bert/mnli/multiruns/2022-05-25/14-54-19/0/checkpoints/last.ckpt"
new = "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/classifier_ablation/3_epoch/runs/2022-06-17/00-43-18/checkpoints/last.ckpt"

In [5]:
old_model = SequenceClassificationTransformer.load_from_checkpoint(old)
new_model = SequenceClassificationTransformer.load_from_checkpoint(new)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bi

In [9]:
torch.allclose(old_model.classifier.weight, new_model.classifier.weight)

True

In [10]:
old_model.classifier.weight, new_model.classifier.weight

(Parameter containing:
 tensor([[-0.0023, -0.0115,  0.0040,  ..., -0.0247,  0.0227, -0.0029],
         [-0.0044,  0.0032, -0.0018,  ...,  0.0419,  0.0060, -0.0034],
         [ 0.0433,  0.0028,  0.0247,  ...,  0.0192, -0.0078, -0.0276]],
        requires_grad=True),
 Parameter containing:
 tensor([[-0.0023, -0.0115,  0.0040,  ..., -0.0247,  0.0227, -0.0029],
         [-0.0044,  0.0032, -0.0018,  ...,  0.0419,  0.0060, -0.0034],
         [ 0.0433,  0.0028,  0.0247,  ...,  0.0192, -0.0078, -0.0276]],
        requires_grad=True))